In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Flatten, Dense, AvgPool2D
from tensorflow.keras import Model
import os
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu)

In [ ]:
import tensorflow_datasets as tfds

# Load the ImageNet 32x32 dataset
dataset_name = "imagenet_resized/32x32"
dataset, info = tfds.load(dataset_name, split=['train', 'validation'], as_supervised=True, with_info=True)

# Split into train and validation sets
train_dataset = dataset[0]
test_dataset = dataset[1]

# Display dataset info
print(info)

In [49]:
image_list, label_list = [], []

for image, label in tfds.as_numpy(dataset[0]):
    image_list.append(image)
    label_list.append(label)

image_array = np.array(image_list)
label_array = np.array(label_list)

In [50]:
# Perform stratified sampling (80% train, 20% validation)
train_images, val_images, train_labels, val_labels = train_test_split(
    image_array, label_array, test_size=0.2, stratify=label_array, random_state=42
)

In [ ]:
# Convert back to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))

In [ ]:
# Preprocess and batch
batch_size = 32
train_dataset = train_dataset.shuffle(len(train_images)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Check dataset sizes
print(f"Train Size: {len(train_images)}, Validation Size: {len(val_images)}")

In [ ]:
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x


input = Input(shape = (32,32,3))
x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)

x = mobilnet_block(x, filters = 64, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 2)
x = mobilnet_block(x, filters = 256, strides = 2)
x = mobilnet_block(x, filters = 512, strides = 1)
x = mobilnet_block(x, filters = 512, strides = 2)
for _ in range (5):
    x = mobilnet_block(x, filters = 512, strides = 1)
x = mobilnet_block(x, filters = 1024, strides = 2)
x = mobilnet_block(x, filters = 1024, strides = 1)
# x = AvgPool2D ()(x)
x = Flatten()(x)
output = Dense (units = 1000, activation = 'softmax')(x)
model = Model(inputs=input, outputs=output)
model.summary()

In [9]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00003),
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [10]:
model_checkpoint = ModelCheckpoint('MobileNet_2.h5', save_best_only=True, monitor="val_loss")

In [ ]:
# Train the model using PrefetchDataset
model.fit(train_dataset, validation_data=val_dataset, epochs=10, callbacks=[model_checkpoint])

In [ ]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00002),
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=[model_checkpoint])

In [24]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.resize_with_crop_or_pad(image, 36, 36)  # Add padding
    image = tf.image.random_crop(image, [32, 32, 3])  # Crop back to 32x32
    return image, label

In [25]:
# Apply augmentation to the dataset
augment_train_dataset = train_dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
augment_train_dataset = augment_train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
# Train the model
training = model.fit(
    augment_train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[model_checkpoint]
)

In [44]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [ ]:

training = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    callbacks=[model_checkpoint]
)

In [53]:
batch_size = 32  # Adjust based on available memory
test_dataset = test_dataset.batch(batch_size)  # Add batching

In [ ]:
model.evaluate(test_dataset)

In [33]:
model.save(os.path.join('models','MobileNet.h5'))